In [1]:
import requests
from bs4 import BeautifulSoup

url = "https://books.toscrape.com/"
all_books = []

while url:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    books = soup.find_all("article", class_="product_pod")
    for book in books:
        title = book.h3.a["title"]
        price = book.find("p", class_="price_color").text
        availability = book.find("p", class_="instock availability").text.strip()
        rating = book.p["class"][1]
        all_books.append((title, price, availability, rating))

    # Find next page
    next_button = soup.find("li", class_="next")
    if next_button:
        next_page = next_button.a["href"]
        url = requests.compat.urljoin(url, next_page)  # Build full URL for next page
    else:
        url = None  # Stop when no more pages

print("Total books scraped:", len(all_books))
print(all_books[:5])


Total books scraped: 1000
[('A Light in the Attic', 'Â£51.77', 'In stock', 'Three'), ('Tipping the Velvet', 'Â£53.74', 'In stock', 'One'), ('Soumission', 'Â£50.10', 'In stock', 'One'), ('Sharp Objects', 'Â£47.82', 'In stock', 'Four'), ('Sapiens: A Brief History of Humankind', 'Â£54.23', 'In stock', 'Five')]


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time

# Start Chrome (Selenium Manager automatically finds driver)
driver = webdriver.Chrome()

# Open IMDB Top 250
driver.get("https://www.imdb.com/chart/top/")

# Wait for page to load
time.sleep(3)

# Find all movie containers
movies = driver.find_elements(By.CSS_SELECTOR, "li.ipc-metadata-list-summary-item")

movie_data = []

for movie in movies:
    # Rank
    rank = movie.find_element(By.CSS_SELECTOR, "span.ipc-title__text").text.split('.')[0]
    
    # Title
    title = movie.find_element(By.CSS_SELECTOR, "h3.ipc-title__text").text.split('. ')[-1]
    
    # Year of release
    year = movie.find_element(By.CSS_SELECTOR, "span.sc-b189961a-8").text
    
    # IMDB Rating
    rating = movie.find_element(By.CSS_SELECTOR, "span.ipc-rating-star--rating").text
    
    movie_data.append({
        "Rank": int(rank),
        "Movie Title": title,
        "Year of Release": year,
        "IMDB Rating": rating
    })

# Create DataFrame
df = pd.DataFrame(movie_data)

# Export to CSV
df.to_csv("imdb_top250.csv", index=False)

print("✅ IMDB Top 250 Movies scraped successfully!")
print(df.head())

# Close browser
driver.quit()


ModuleNotFoundError: No module named 'selenium'

In [5]:
!pip install selenium


Access is denied.


In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.timeanddate.com/weather/"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# Try to find the main table dynamically
table = soup.find("table")
if not table:
    print("❌ Could not find the weather table on the page.")
else:
    cities_data = []

    # Each row represents one city (skip header)
    for row in table.find_all("tr")[1:]:
        cols = row.find_all("td")
        if len(cols) >= 3:
            city_name = cols[0].text.strip()
            temperature = cols[1].text.strip()
            condition = cols[2].text.strip()
            cities_data.append({
                "City Name": city_name,
                "Temperature": temperature,
                "Weather Condition": condition
            })

    # Convert to DataFrame
    df = pd.DataFrame(cities_data)

    # Export to CSV
    df.to_csv("weather.csv", index=False)

    print("✅ Weather data scraped successfully!")
    print(df.head())


✅ Weather data scraped successfully!
     City Name  Temperature Weather Condition
0        Accra  शुक्र 03.58                  
1  Addis Ababa  शुक्र 06.58                  
2   Adelaide *  शुक्र 14.28                  
3      Algiers  शुक्र 04.58                  
4       Almaty  शुक्र 08.58                  
